# Gaussian Naive Bayes Model

In [1]:
import sklearn
sklearn.__version__

'0.23.1'

In [2]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import time
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn import ensemble
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import linear_model
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics.pairwise import euclidean_distances

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

## Read the provided dataset and understand the observations

In [3]:
# Read data set provided from the Kaggle competition website

train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

print("The training dataset contains {0} observations with {1} features for each observation.".\
    format(train_df.shape[0], train_df.shape[1]))
train_df.head()

The training dataset contains 15120 observations with 56 features for each observation.


,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


## Split the training data into train and dev dataset

In [4]:
# Split the training data into train and dev data set.
# Read the training data into X and y
train_file = open("data/train.csv")
column_names_train = train_file.readline()
data = np.loadtxt(train_file, delimiter=",")

y, X = data[:, -1].astype('u1'), data[:, :-1]

# Shuffle the data, but make sure that the features and accompanying labels stay in sync.
np.random.seed(0)
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, y = X[shuffle], y[shuffle]

# Split the training data into 90% training data and 10% dev data
train_size = int(X.shape[0] * 0.9)

# Discard 1st feature (ID number that doesn't provide info about the label)
y_train, X_train = y[:train_size], X[:train_size, 1:]
y_dev, X_dev = y[train_size:], X[train_size:, 1:]
print(X_dev.shape, X_train.shape)

# Read the test data and store in X_test
test_file = open("data/test.csv")
column_names_test = test_file.readline()
data_test = np.loadtxt(test_file, delimiter=",")

# Save the test data in X_test. Test data does not have the 1st feature
X_test = data_test
X_test = X_test[:, 1:]
print(X_test.shape)

(1512, 54) (13608, 54)
(565892, 54)


In [8]:
# import training data from relative filepath
data = pd.read_csv("data/train.csv")

# extract training data except labels and ID column
train_df = data.loc[:, (data.columns != "Cover_Type") & (data.columns != "Id")]

# extract labels from training data
train_labels_df = data.loc[:, "Cover_Type"]

# import test data from relative filepath
test_data = pd.read_csv("data/test.csv")

# extract test data except ID column
test_df = test_data.loc[:, test_data.columns != "Id"]

# train model using K-NN and the mini_train data and mini_train_labels
knn_model = KNeighborsClassifier(n_neighbors=1, metric='euclidean')
knn_model.fit(train_df, train_labels_df)

# Supply the test_df to knn_model and create predictions
predictions = knn_model.predict(test_df)

# converts predictions from np array to pd dataframe
predictions_df = pd.DataFrame(data = predictions, index = test_data.loc[:, "Id"], columns = ["Cover_Type"])

predictions_df

,Cover_Type
Id,
15121,2
15122,2
15123,1
15124,1
15125,1
...,...
581008,3
581009,3
581010,3


# Gaussian Naive Bayes Model

In [5]:
# Develop Gaussian Naive Bayes model

'''Gaussian Naive Bayes is a variant of Naive Bayes that follows Gaussian normal distribution 
and supports continuous data'''

GNB_model = GaussianNB()
GNB_model.fit(X_train[:,:10], y_train)

# Predict the labels by passing the dev data that was split out from train data
dev_predicted_labels = GNB_model.predict(X_dev[:,:10])

# Print accuracy scores
print(metrics.accuracy_score(y_true=y_dev, y_pred=dev_predicted_labels))
print(metrics.classification_report(y_dev, dev_predicted_labels))

0.5906084656084656
              precision    recall  f1-score   support

           1       0.55      0.56      0.55       220
           2       0.50      0.42      0.46       208
           3       0.50      0.40      0.44       220
           4       0.71      0.78      0.74       212
           5       0.62      0.69      0.65       227
           6       0.45      0.47      0.46       206
           7       0.75      0.80      0.78       219

    accuracy                           0.59      1512
   macro avg       0.58      0.59      0.58      1512
weighted avg       0.58      0.59      0.58      1512



In [9]:
# Develop Gaussian Naive Bayes model
GNB_model = GaussianNB()
GNB_model.fit(train_df, train_labels_df)

# Supply the test_df to GNB_model and create predictions
predictions = GNB_model.predict(test_df)

# converts predictions from np array to pd dataframe
predictions_NB_df = pd.DataFrame(data = predictions, index = test_data.loc[:, "Id"], columns = ["Cover_Type"])

predictions_NB_df

,Cover_Type
Id,
15121,2
15122,2
15123,1
15124,2
15125,2
...,...
581008,3
581009,3
581010,3


In [10]:
# outputs to csv file
predictions_NB_df.to_csv("nb_predictions.csv")

# When submitted into kaggle, it provided a score of 0.42149 much lower than K-NN